In [ ]:
import pandas as pd
import numpy as np

# Score Matrix (Base Score + Harmony Score)
scores = np.array(([[5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1]], # main good
                  [[5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1]], # CG for MG 1
                  [[5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1]], #cg for MG 2
                  [[5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1]], # cg for mg 3
                  [[5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1]], # cg for mg 4
                  [[5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1], [5,4,3,2,1]])) #cg for mg 5

In [ ]:
class ScoreCalculator:
    def __init__(self):
        self.form_1_player_details = None
        self.form_1_player_responses = None

    def convert_data(self, path):
        data = pd.read_csv(path)
        player_detail_columns = ["Team Leader\'s Name", "Contact Number (Team Leader)", "Email Id (Team Leader)", "Final Score for MG"]
        player_details = data[player_detail_columns]
        player_responses = data[data.drop(player_detail_columns, axis=1).columns]
        return player_details, player_responses

    def import_data(self, path):
        self.form_1_player_details, self.form_1_player_responses = self.convert_data(path)

    def MG_calculator(self, player_details, player_responses, MGR_column_names):

      final_scores_list = []

      MGR = player_responses[MGR_column_names]

      for i in range(len(player_responses)):
          detail = player_details.iloc[i]
          response = player_responses.iloc[i]

          n_questions = 6
          n_options = 5

          sparse_matrix = np.zeros((n_questions, n_options), dtype=int)
          for idx, answer in enumerate(response):
              if 1 <= answer <= n_options:
                  sparse_matrix[idx, answer - 1] = 1
              else:
                  print("Error: Invalid response detected")

          options_with_base_score = np.multiply(sparse_matrix, self.scores[:, :, 0])
          options_with_base_score_and_importance = options_with_base_score * self.spot_importance[:, np.newaxis]
          final_score = np.sum(options_with_base_score_and_importance)

          final_scores_list.append((detail.iloc[0], final_score))

      scores_df = pd.DataFrame(final_scores_list, columns=["Team Name", "Team Leader Name", "Phone Number", "Final Score for CG"])
      return scores_df

    def CG_calculator(self, player_details, player_responses, MGR_column_names, CGR_column_names):
      final_scores_list = []
      spot_importance = np.array([1, 1, 1, 1, 1, 1])

      MGR = player_responses[MGR_column_names]
      CGR = player_responses[CGR_column_names]

      for i in range(len(CGR)):
          detail = player_details.iloc[i]
          CGR_i = CGR.iloc[i]
          MGR_i = MGR.iloc[i]

          sparse_matrix = np.zeros((6, 5), dtype=int)
          for idx, answer in enumerate(CGR_i):
              if 1 <= answer <= 5:
                  sparse_matrix[idx, answer - 1] = 1
              else:
                  print("Error: Invalid response detected")

          for q in range(len(CGR_i)):
              sparse_matrix[q, :] *= self.scores[q, :, MGR_i[q]]

          sparse_matrix *= spot_importance[:, np.newaxis]
          final_score = np.sum(sparse_matrix)

          final_scores_list.append((detail.iloc[0], final_score))

      CG_scores_df = pd.DataFrame(final_scores_list, columns=["Team Name", "Team Leader Name", "Phone Number", "Final Score for CG"])
      return CG_scores_df

    def score_calculator(self, player_details, player_responses, MGR_column_names, CGR_column_names):

        MG_scores = self.MG_calculator(player_details, player_responses, MGR_column_names)
        CG_scores = self.CG_calculator(player_details, player_responses, MGR_column_names, CGR_column_names)

        if not MG_scores["Team Name"].equals(CG_scores["Team Name"]):
            raise ValueError("Player details do not match across MG and CG calculations")

        final_score = pd.concat([MG_scores, CG_scores], axis=1)
        return final_score

    def final_wala_calculator(self, MGR_column_names, CGR_column_names):

        one = self.score_calculator(self.form_1_player_details, self.form_1_player_responses, MGR_column_names, CGR_column_names)
        two = self.score_calculator(self.form_2_player_details, self.form_2_player_responses, MGR_column_names, CGR_column_names)
        three = self.score_calculator(self.form_3_player_details, self.form_3_player_responses, MGR_column_names, CGR_column_names)

        final_scores = pd.concat([one, two, three], axis=1)
        return final_scores

In [ ]:
calculator = ScoreCalculator()

In [ ]:
calculator.import_data(['/content/drive/MyDrive/bequest/form_1.csv', '/content/drive/MyDrive/bequest/form_2.csv', '/content/drive/MyDrive/bequest/form_3.csv'])

In [ ]:
scores = calculator.final_wala_calculator(['Spot 1', 'Spot 2', 'Spot 3', 'Spot 4', 'Spot 5', 'Spot 6'], ['CG Spot 1',
       'CG Spot 2', 'CG Spot 3', 'CG Spot 4', 'CG Spot 5', 'CG Spot 6'])

In [ ]:
calculator.form_1_player_responses.columns